Fix missing publication year

1. Get year from academic analytics
2. write to local json
3. push to milvus


In [ ]:
from pathlib import Path

AUTHORS_DIR = Path("data/raw/campus_users/")
print(f"n={len(list(Path(AUTHORS_DIR).glob('*.json')))}")

In [ ]:
from embedding_search.academic_analytics import *

In [ ]:
a = get_author(277399)

In [ ]:
a["articles"]

In [ ]:
from crawl import download_one_author

download_one_author(4575)

In [ ]:
from datetime import datetime

datetime.today().strftime("%Y-%m-%d")

In [ ]:
u = get_units()

In [ ]:
f = get_faculties(unit_id=28574)

In [ ]:
f[0]

In [ ]:
get_articles(277399)

In [ ]:
a[0]["journalYear"]

In [ ]:
id = 277399
url = f"https://wisc.discovery.academicanalytics.com/api/people/{id}"
response = requests.get(url)

In [ ]:
data = response.json()

In [ ]:
data.keys()

In [ ]:
data["firstName"].title()

In [ ]:
data["lastName"]

In [ ]:
data["primaryUnitAffiliation"]["id"]

In [ ]:
data["articles"]

In [ ]:
import requests
from time import sleep

dois = [
    "10.1089/JPM.2022.0036",
    "10.1007/S00520-021-06399-Z",
    "10.25159/1947-9417/13316",
]

from embedding_search.crossref import batch_query_cited_by

y = batch_query_cited_by(dois)

In [ ]:
y

In [ ]:
output

In [ ]:
data["message"]

In [ ]:
data["message"]

In [ ]:
data["message"]

In [ ]:
response = requests.get(url, params=params)
data = response.json()

In [ ]:
data["message"].get("next-cursor")

In [ ]:
# Make a dictionary of {"doi": {"citation_count": n, "year": y}}

import json
import os
import re
from collections import defaultdict
from functools import cache
from pathlib import Path
from typing import Any, DefaultDict, Dict, List, Union

import pandas as pd

In [ ]:
for item in data["message"]["items"]:
    print("DOI:", item["DOI"])
    print("Citation Count:", item["is-referenced-by-count"])
    print("Publication Year:", item["created"]["date-parts"][0][0])